# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [1]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp


## Loading data

In [2]:
dataset = load_adult()
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [3]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,0.833333,0.043338,0.000000,0.800000,0.333333,0.615385,0.6,0.0,1.0,0.02174,0.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.047277,0.000000,0.800000,0.000000,0.307692,0.4,0.0,1.0,0.00000,0.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.137244,0.200000,0.533333,0.166667,0.461538,0.6,0.0,1.0,0.00000,0.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.150212,0.133333,0.400000,0.000000,0.461538,0.4,1.0,1.0,0.00000,0.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.220703,0.000000,0.800000,0.000000,0.384615,0.0,1.0,0.0,0.00000,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [4]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 28162
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [5]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [6]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [7]:
X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
base_clf = MLPClassifier()
base_clf.fit(X_train, y_train)

def eval_model(dataset_train, dataset_test, use_base_clf=True):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    if use_base_clf:
        y_train = base_clf.predict(X_train)

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [8]:
eval_model(dataset_train, dataset_test, use_base_clf=False)

{'precision': 0.8807397959183674,
 'recall': 0.9194407456724367,
 'auroc': 0.7719693688201541,
 'dp': 0.18931053047861435,
 'ftu': 0.04499999999999993}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [9]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

2022-01-27 10:32:23.688771: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-27 10:32:23.721846: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Synthetic data generation: 100%|██████████| 221/221 [00:02<00:00, 86.59it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.000980,0.166667,0.002408,0.133333,0.066667,0.166667,0.615385,0.4,0.25,1.0,0.000108,0.004265,-0.017734,0.350,0.0
1,0.003079,0.500000,0.003174,0.266667,0.866667,0.000000,1.000000,0.6,0.50,0.0,-0.000117,0.004390,-0.014888,0.275,1.0
2,0.006672,0.000000,0.004719,0.733333,0.000000,0.500000,0.307692,0.6,0.00,0.0,-0.003958,0.005392,-0.010916,0.425,1.0
3,0.002382,0.000000,0.003590,0.866667,0.133333,0.166667,0.692308,0.4,1.00,1.0,-0.001449,0.003632,-0.017750,0.250,0.0
4,0.002255,0.166667,0.002813,0.866667,0.666667,0.500000,0.230769,0.6,0.25,0.0,0.000522,0.004543,-0.015989,0.750,0.0


In [10]:
eval_model(synth_data, dataset_test)

{'precision': 0.751, 'recall': 1.0, 'auroc': 0.5, 'dp': 0.0, 'ftu': 0.0}

### WGAN-GP

In [11]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

Synthetic data generation: 100%|██████████| 57/57 [00:01<00:00, 53.26it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.004468,0.333333,0.133133,0.666667,0.533333,0.833333,0.307692,0.0,1.00,0.0,0.010781,-0.020771,0.339623,0.800,0.0
1,0.104653,0.833333,0.124953,0.600000,0.400000,0.833333,0.769231,0.0,0.75,0.0,0.010670,0.011339,0.327490,0.625,1.0
2,0.093025,0.833333,0.093451,0.333333,0.533333,0.000000,1.000000,0.6,0.75,1.0,0.011444,-0.004764,0.328748,0.625,1.0
3,0.185383,0.666667,0.214539,1.000000,0.333333,1.000000,0.846154,1.0,1.00,0.0,0.002735,0.061225,0.432093,0.875,1.0
4,0.001776,0.500000,0.199525,0.066667,0.666667,0.000000,0.615385,1.0,0.00,1.0,-0.003021,-0.010334,0.396284,1.000,1.0


In [12]:
eval_model(synth_data, dataset_test)

{'precision': 0.8296382730455076,
 'recall': 0.9467376830892144,
 'auroc': 0.6801961507815549,
 'dp': 0.10024577583544847,
 'ftu': 0.01550000000000007}

### FairGAN

In [13]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

INFO:tensorflow:Restoring parameters from cache/fairgan
burning in


2022-01-27 10:32:56.356939: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


generating


/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.608476,0.010220,1.125662,0.368530,0.000000,0.500347,0.204989,0.000000,0.317873,0.0,0.348013,0.285200,0.000000,0.986342,1.0
1,0.000000,0.164930,0.465049,0.385007,0.200864,0.180325,0.566214,0.000000,0.862392,0.0,0.000000,0.431988,0.608271,0.516321,1.0
2,0.217014,0.188261,0.370919,0.886405,0.052378,0.446725,0.572333,0.000000,0.487659,0.0,0.156805,0.542946,0.000000,0.000000,1.0
3,0.000000,0.094479,0.493281,0.603377,0.241376,0.304915,0.593466,0.000000,0.478712,0.0,0.000000,0.411942,0.000000,0.996604,1.0
4,0.190785,0.242342,0.520125,0.477219,0.000000,0.418284,0.317209,0.129395,0.323773,0.0,0.000000,0.386664,0.000000,0.912659,1.0


In [14]:
eval_model(synth_data, dataset_test, use_base_clf=False)

{'precision': 0.7648578811369509,
 'recall': 0.9853528628495339,
 'auroc': 0.5358491221878192,
 'dp': 0.08074097798349011,
 'ftu': 0.1725}

### DECAF

#### DECAF-ND

In [15]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
17812,0.191291,5.175647e-04,0.018231,0.083824,0.636673,0.514681,0.103576,0.960376,0.000000e+00,0.000498,7.315889e-09,1.618124e-12,0.229568,2.466159e-27,1.0
7446,0.218342,1.941178e-03,0.106094,0.084492,0.554245,0.021609,0.194821,0.062071,1.288539e-24,0.000470,1.733381e-14,1.229649e-09,0.597386,6.414558e-15,0.0
3675,0.063070,3.312119e-06,0.028163,0.058360,0.555362,0.284339,0.610718,0.370714,0.000000e+00,0.000575,2.333810e-14,4.014825e-05,0.179631,6.424188e-04,1.0
31448,0.139535,1.421372e-09,0.069024,0.057666,0.552814,0.302833,0.135508,0.496631,5.120023e-09,0.000684,8.526458e-06,3.610539e-08,0.550769,1.403233e-24,0.0
20706,0.106182,1.029702e-04,0.043004,0.101137,0.993224,0.292178,0.467095,0.657944,8.388181e-22,0.000532,1.698673e-13,1.775935e-09,0.177355,1.223973e-07,1.0


In [16]:
eval_model(synth_data, dataset_test)

{'precision': 0.8436213991769548,
 'recall': 0.9553928095872171,
 'auroc': 0.7106281317012391,
 'dp': 0.13047471126218124,
 'ftu': 0.02300000000000002}

#### DECAF-FTU

In [17]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
17812,0.412485,2.545034e-07,0.173560,0.080902,0.556328,4.915766e-01,0.534028,0.275533,0.000000e+00,0.000471,1.292753e-13,1.769474e-15,0.131490,3.172505e-06,1.0
7446,0.094101,2.074368e-02,0.287957,0.110273,0.556262,3.454367e-01,0.209828,0.500177,7.901006e-01,0.000467,3.292900e-07,3.116820e-01,0.323860,2.001139e-25,1.0
3675,0.551142,8.912112e-05,0.085676,0.249329,0.555636,2.817445e-07,0.719045,0.394943,3.428525e-26,1.000000,6.444314e-13,1.096126e-10,0.170164,2.663476e-01,1.0
31448,0.531663,7.150981e-01,0.011443,0.110235,0.556089,4.156794e-01,0.123529,0.908106,0.000000e+00,1.000000,2.837682e-12,1.745657e-18,0.368959,3.357129e-07,0.0
20706,0.551893,2.783317e-07,0.066595,0.116750,0.553694,5.817314e-03,0.267963,0.323046,6.917830e-28,0.000791,1.466752e-08,1.073257e-11,0.171665,8.356533e-29,0.0


In [18]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8463802236609771,
 'recall': 0.9573901464713716,
 'auroc': 0.7166468804646015,
 'dp': 0.127354586457189,
 'ftu': 0.023499999999999965}

#### DECAF-CF

In [19]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
17812,0.540873,3.087743e-01,0.018888,0.062059,0.553097,0.008951,0.399138,0.872691,9.872845e-21,0.000510,1.783311e-12,3.391629e-05,0.365252,2.050162e-14,1.0
7446,0.152107,2.730139e-01,0.094206,0.233198,0.542134,0.483485,0.434027,0.316910,0.000000e+00,0.000733,7.757101e-05,2.383838e-18,0.125056,8.880838e-21,1.0
3675,0.035499,3.372478e-09,0.091518,0.120004,0.552442,0.010240,0.288018,0.417964,9.913976e-01,1.000000,7.356760e-03,3.918837e-23,0.150271,1.942173e-06,1.0
31448,0.102030,3.001635e-08,0.020693,0.184442,0.517513,0.374050,0.762049,0.260376,0.000000e+00,0.999998,4.585064e-08,1.131372e-07,0.245905,5.852722e-22,1.0
20706,0.458230,1.249771e-03,0.106744,0.181281,0.934044,0.378368,0.563302,0.918130,7.659550e-38,0.000467,7.838521e-06,1.158475e-19,0.474347,2.362235e-22,0.0


In [20]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8451118963486455,
 'recall': 0.9553928095872171,
 'auroc': 0.7136401798940102,
 'dp': 0.12579452405469294,
 'ftu': 0.0014999999999999458}

#### DECAF-DP

In [21]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_dp)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
17812,0.551911,0.406031,0.020041,0.065765,0.555935,4.345795e-01,0.410596,0.300615,5.678213e-31,0.000479,1.252516e-13,1.231502e-04,0.350512,5.360784e-07,1.0
7446,0.476455,0.000001,0.229373,0.195291,0.554193,3.748886e-07,0.461938,0.392525,2.621925e-31,1.000000,4.284466e-02,2.592287e-09,0.667083,4.768144e-04,0.0
3675,0.117431,0.723073,0.061697,0.189970,0.555362,3.075788e-01,0.076947,0.238301,0.000000e+00,1.000000,2.068684e-13,3.545676e-06,0.320052,2.176982e-21,1.0
31448,0.072518,0.379646,0.052438,0.170150,0.996842,5.390463e-04,0.703045,0.322105,4.776207e-18,1.000000,3.690036e-04,1.084158e-13,0.350855,4.176400e-25,0.0
20706,0.034426,0.000032,0.094144,0.103669,0.555231,2.933795e-03,0.075759,0.415128,0.000000e+00,1.000000,6.980293e-07,1.015996e-14,0.230010,3.355400e-23,1.0


In [22]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.853309481216458,
 'recall': 0.9527296937416777,
 'auroc': 0.7293768950636099,
 'dp': 0.1389511651106221,
 'ftu': 0.007500000000000062}